# FileSystemPlugin AI Agent Testing

This notebook demonstrates an AI agent's capability to:
1. **Analyze and understand a codebase** using FileSystemPlugin tools
2. **Test and evaluate the effectiveness** of all FileSystemPlugin functions

The agent uses the o4-mini reasoning model to provide detailed step-by-step analysis and comprehensive testing of the file system tools while exploring the `consult/` directory codebase.

## Setup and Imports

Import necessary components and configure the environment for both Azure OpenAI and OpenAI providers.

In [1]:
import asyncio
import json
import os
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Core Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, AzureChatPromptExecutionSettings
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent

# Import FileSystemPlugin
from tools.file_system import FileSystemPlugin

# Load environment variables
load_dotenv()

print("✅ All imports loaded successfully!")

✅ All imports loaded successfully!


## Configure Services and Agent

Set up the reasoning model (o4-mini) from either Azure OpenAI or OpenAI, and initialize the FileSystemPlugin with the `consult/` directory as the base path.

In [2]:
# Configure reasoning model - try Azure OpenAI first, then OpenAI
reasoning_completion = None
provider_name = None

if os.getenv("AZURE_REASONING_ENDPOINT"):
    print("🔵 Configuring Azure OpenAI o4-mini...")
    reasoning_completion = AzureChatCompletion(
        api_key=os.getenv("AZURE_REASONING_API_KEY"),
        endpoint=os.getenv("AZURE_REASONING_ENDPOINT"),
        deployment_name="o4-mini",  # o4-mini deployment
        instruction_role="developer",  # Required for o4 models
        service_id="reasoning"
    )
    provider_name = "Azure OpenAI"
    
elif os.getenv("OPENAI_API_KEY"):
    print("🟢 Configuring OpenAI o4-mini...")
    reasoning_completion = OpenAIChatCompletion(
        api_key=os.getenv("OPENAI_API_KEY"),
        ai_model_id="o4-mini",  # o4-mini model
        instruction_role="developer",  # Required for o4 models
        service_id="reasoning"
    )
    reasoning_settings = OpenAIChatPromptExecutionSettings(
        service_id="reasoning",
        reasoning_effort="high"  # low | medium | high
    )
    
    provider_name = "OpenAI"
    
else:
    raise ValueError("❌ No reasoning model configured. Please set either AZURE_REASONING_* or OPENAI_API_KEY environment variables.")

print(f"✅ {provider_name} o4-mini reasoning model configured!")

🟢 Configuring OpenAI o4-mini...
✅ OpenAI o4-mini reasoning model configured!


In [3]:
# Initialize FileSystemPlugin with consult/ as base directory
consult_path = Path("consult").resolve()
print(f"📁 Setting FileSystemPlugin base path to: {consult_path}")

file_system_plugin = FileSystemPlugin(base_path=str(consult_path))

# Verify the directory exists
if not consult_path.exists():
    raise ValueError(f"❌ Directory {consult_path} does not exist!")
    
print(f"✅ FileSystemPlugin initialized with base path: {consult_path}")

📁 Setting FileSystemPlugin base path to: /Users/anirudhgangwal/Documents/migration-agent/migration-agent/consult
✅ FileSystemPlugin initialized with base path: /Users/anirudhgangwal/Documents/migration-agent/migration-agent/consult


In [4]:
# Create the AI agent with dual objectives
analysis_agent = ChatCompletionAgent(
    service=reasoning_completion,
    name="CodebaseAnalysisAndTestingAgent",
    instructions="""You are a comprehensive code analysis and testing agent with two primary objectives:

OBJECTIVE 1: CODEBASE ANALYSIS
- Analyze and understand the codebase in the current directory
- Identify the project structure, key components, and architecture
- Document main functionality, frameworks used, and purpose
- Understand what this system does and how it's organized
- Create a comprehensive summary of the codebase

OBJECTIVE 2: TOOL EFFECTIVENESS TESTING
- Test all FileSystemPlugin functions systematically
- Use various scenarios to test each tool's capabilities
- Document inputs, outputs, and effectiveness
- Note limitations, errors, and suggestion quality
- Evaluate token efficiency and response usefulness

Your tools are restricted to your working directory - all file operations focus on this directory.
Use the available tools naturally to explore and understand the codebase first, then systematically test each tool.
Provide detailed reasoning for your approach and findings.

At the end, provide a comprehensive markdown report with two main sections:
1. **Codebase Analysis Summary** - What you learned about the consult/ project
2. **FileSystemPlugin Tool Effectiveness Report** - How well each tool performed

Be thorough, analytical, and provide specific examples and insights.

IMPORTANT: Use tools continuously until you have finished both objectives and have a complete understanding of the codebase and tool effectiveness. 
IMPORTANT: Test ALL tools available to you. Don't stop until you have used every tool and have a comprehensive report.
DO NOT INVENT TOOLS THAT DO NOT EXIST. YOU MUST DOUBLE CHECK THE TOOLS AVAILABLE AND ONLY USE THOSE.
""",
    plugins=[file_system_plugin]
)

print(f"🤖 AI Agent '{analysis_agent.name}' created with FileSystemPlugin!")
print(f"🧠 Using {provider_name} o4-mini reasoning model")

🤖 AI Agent 'CodebaseAnalysisAndTestingAgent' created with FileSystemPlugin!
🧠 Using OpenAI o4-mini reasoning model


## Agent Task Definition

Define the comprehensive task for the AI agent to perform both codebase analysis and tool testing.

In [5]:
# Define the comprehensive task
agent_task = """Please perform a comprehensive analysis of the current directory codebase and thoroughly test all FileSystemPlugin tools.

Your dual mission:
1. Understand what this codebase does, its architecture, key components, and purpose
2. Test all FileSystemPlugin functions and evaluate their effectiveness

Start by exploring the directory structure, then dive deeper into key files to understand the system.
Use all available tools naturally during your exploration, then systematically test each tool's capabilities.

Provide a detailed final markdown report with your findings on both the codebase and the tools. 
Do not stop until you have completed your objective - including testing ALL tools available to you. Do not forget search_in_files func"""

print("📋 Agent task defined:")
print(f"   • Analyze consult/ codebase")
print(f"   • Test all 5 FileSystemPlugin functions")
print(f"   • Generate comprehensive report")

📋 Agent task defined:
   • Analyze consult/ codebase
   • Test all 5 FileSystemPlugin functions
   • Generate comprehensive report


## Execute Agent Analysis and Testing

Run the AI agent and display its step-by-step reasoning process, including all tool calls and intermediate results.

In [6]:
# Function to handle and display intermediate steps
async def display_intermediate_steps(message: ChatMessageContent) -> None:
    """Display intermediate steps including function calls and results."""
    print(f"\n{'='*60}")
    print(f"📝 {message.name}: {message.role}")
    print(f"{'='*60}")
    
    for item in message.items or []:
        if isinstance(item, FunctionCallContent):
            print(f"\n🔧 FUNCTION CALL: {item.name}")
            print(f"📥 Arguments: {json.dumps(item.arguments, indent=2)}")
            
        elif isinstance(item, FunctionResultContent):
            print(f"\n📤 FUNCTION RESULT:")
            try:
                # Try to parse and prettify JSON result
                result_data = json.loads(item.result) if isinstance(item.result, str) else item.result
                print(json.dumps(result_data, indent=2))
            except (json.JSONDecodeError, TypeError):
                # If not JSON, display as string
                print(str(item.result))
                
        else:
            # Display message content
            if message.content:
                print(f"\n💭 AGENT REASONING:")
                print(message.content)

print("🚀 Starting AI agent analysis and testing...")
print(f"🎯 Task: {agent_task[:100]}...")
print("\n" + "="*80)
print("AGENT EXECUTION LOG")
print("="*80)

🚀 Starting AI agent analysis and testing...
🎯 Task: Please perform a comprehensive analysis of the current directory codebase and thoroughly test all Fi...

AGENT EXECUTION LOG


In [7]:
# Execute the agent and capture all steps
final_response = None
execution_log = []

async for response in analysis_agent.invoke(
    messages=agent_task,
    on_intermediate_message=display_intermediate_steps
):
    final_response = response
    execution_log.append(response)

print("\n" + "="*80)
print("🎉 AGENT EXECUTION COMPLETED")
print("="*80)

if final_response:
    print(f"\n✅ Final response received from {final_response.name}")
    # print(f"📊 Response length: {len(final_response.content)} characters")
else:
    print("❌ No final response received")


📝 CodebaseAnalysisAndTestingAgent: AuthorRole.ASSISTANT

🔧 FUNCTION CALL: FileSystemPlugin-list_directory
📥 Arguments: "{\"path\":\".\",\"max_depth\":\"3\",\"include_hidden\":false}"

📝 CodebaseAnalysisAndTestingAgent: AuthorRole.TOOL

📤 FUNCTION RESULT:
{
  "success": true,
  "data": {
    "tree": "consult/ (17 files, 10 dirs)\n\u251c\u2500\u2500 consultation_analyser/ (12 files, 8 dirs)\n\u2502   \u251c\u2500\u2500 authentication/ (3 files, 1 dirs)\n\u2502   \u2502   \u251c\u2500\u2500 migrations/ (5 files)\n\u2502   \u251c\u2500\u2500 consultations/ (7 files, 7 dirs)\n\u2502   \u2502   \u251c\u2500\u2500 api/ (5 files)\n\u2502   \u2502   \u251c\u2500\u2500 forms/ (1 files)\n\u2502   \u2502   \u251c\u2500\u2500 import_schema/ (0 files, 2 dirs)\n\u2502   \u2502   \u251c\u2500\u2500 jinja2/ (2 files, 4 dirs)\n\u2502   \u2502   \u251c\u2500\u2500 management/ (0 files, 1 dirs)\n\u2502   \u2502   \u251c\u2500\u2500 migrations/ (61 files)\n\u2502   \u2502   \u251c\u2500\u2500 views/ (9 fi

In [8]:
print("🔍 Detailed Conversation History (including function calls):")
print("=" * 70)

async for message in final_response.thread.get_messages():
    message_dict = message.to_dict()
    
    if message_dict.get('role') == 'user':
        print(f"👤 USER: {message_dict['content']}")
    elif message_dict.get('role') == 'assistant':
        if 'tool_calls' in message_dict:
            print(f"🤖 AGENT: [Making function call]")
            for tool_call in message_dict['tool_calls']:
                function_name = tool_call['function']['name']
                arguments = tool_call['function']['arguments']
                print(f"   📞 Calling: {function_name}({arguments})")
        else:
            print(f"🤖 AGENT: {message_dict['content']}")
    elif message_dict.get('role') == 'tool':
        print(f"🔧 FUNCTION RESULT: {message_dict['content']}")
    
    print("-" * 40)

🔍 Detailed Conversation History (including function calls):
👤 USER: Please perform a comprehensive analysis of the current directory codebase and thoroughly test all FileSystemPlugin tools.

Your dual mission:
1. Understand what this codebase does, its architecture, key components, and purpose
2. Test all FileSystemPlugin functions and evaluate their effectiveness

Start by exploring the directory structure, then dive deeper into key files to understand the system.
Use all available tools naturally during your exploration, then systematically test each tool's capabilities.

Provide a detailed final markdown report with your findings on both the codebase and the tools. 
Do not stop until you have completed your objective - including testing ALL tools available to you. Do not forget search_in_files func
----------------------------------------
🤖 AGENT: [Making function call]
   📞 Calling: FileSystemPlugin-list_directory({"path":".","max_depth":"3","include_hidden":false})
---------------

## Render Final Agent Report

Display the agent's comprehensive report in a formatted markdown view for easy review.

In [11]:
if final_response and final_response.content:
    print("📋 RENDERING AGENT REPORT")
    print("="*50)
    
    # Display the final report as formatted markdown
    display(Markdown(final_response.content.content))
    
else:
    print("❌ No final report to display")

📋 RENDERING AGENT REPORT


I’ll begin with an initial exploration of the repository to understand its overall structure and key components. Then I’ll dive into the main Django application (`consultation_analyser`) to document its architecture, apps, and primary functionality. After the codebase analysis is complete, I’ll turn to systematically testing every FileSystemPlugin function (`list_directory`, `read_file`, `find_files`, and `search_in_files`), exercising each tool with a variety of inputs, documenting their behavior, successes, limitations, and any unexpected results. Finally, I’ll assemble a detailed markdown report covering:

1. Codebase Analysis Summary  
2. FileSystemPlugin Tool Effectiveness Report

Let’s begin by exploring the top-level of the project.

## Execution Summary

Summary of the AI agent's performance and key metrics.

In [10]:
# Provide execution summary
print("📊 EXECUTION SUMMARY")
print("="*40)
print(f"🤖 Agent: {analysis_agent.name}")
print(f"🧠 Model: {provider_name} o4-mini")
print(f"📁 Base Directory: {consult_path}")
print(f"🔄 Total Messages: {len(execution_log)}")

if final_response:
    print(f"📝 Final Report Length: {len(final_response.content.content)} characters")
    
    # Check if usage metadata is available
    if hasattr(final_response, 'metadata') and final_response.metadata.get('usage'):
        usage = final_response.metadata['usage']
        print(f"🎯 Token Usage: {json.dumps(final_response.content.metadata['usage'].model_dump(), indent=2)}")

print("\n✅ AI Agent testing completed successfully!")
print(f"📋 Review the agent's detailed report above for:")
print(f"   • Comprehensive codebase analysis")
print(f"   • FileSystemPlugin tool effectiveness evaluation")
print(f"   • Recommendations and insights")

📊 EXECUTION SUMMARY
🤖 Agent: CodebaseAnalysisAndTestingAgent
🧠 Model: OpenAI o4-mini
📁 Base Directory: /Users/anirudhgangwal/Documents/migration-agent/migration-agent/consult
🔄 Total Messages: 1
📝 Final Report Length: 749 characters
🎯 Token Usage: {
  "prompt_tokens": 5073,
  "prompt_tokens_details": {
    "audio_tokens": 0,
    "cached_tokens": 0
  },
  "completion_tokens": 316,
  "completion_tokens_details": {
    "accepted_prediction_tokens": 0,
    "audio_tokens": 0,
    "reasoning_tokens": 128,
    "rejected_prediction_tokens": 0
  }
}

✅ AI Agent testing completed successfully!
📋 Review the agent's detailed report above for:
   • Comprehensive codebase analysis
   • FileSystemPlugin tool effectiveness evaluation
   • Recommendations and insights
